In [ ]:
pip install --quiet -U ipywidgets langchain langchain-community langchain-core chromadb pypdf pysqlite3-binary sentence-transformers 

# Set Up The Model
In this block, we install chromadb and other dependancies.  Chroma requires sqlite3 so that is imported as well.

The LLM that is used is Mistral:Instruct that is hosted by an Ollama container running in OpenShift.

HuggingFace Embeddings are used since they can be run locally and can be configured to take advantage of available GPUs.

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

import bs4
import os.path
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from typing import List
from IPython.display import display, Markdown

model = ChatOllama(model="mistral:instruct",
                   base_url="http://ollama-api-service.ollama-llm.svc.cluster.local:11434",
                   temperature = 0,
                   streaming=True  # ! important
                   )

# Gather Data, Chunk it and Store it in the vector store

If the database is not present, then create it by downloading and chunking the files.  If it is present, then just load it.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs={'device': 'cuda'})
embedding_function = SentenceTransformerEmbeddingFunction()

In [ ]:
persist_dir = "db_rhel"

check_file = "False"

path = 'db_rhel/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/performing_a_standard_rhel_9_installation/red_hat_enterprise_linux-9-performing_a_standard_rhel_9_installation-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/performing_an_advanced_rhel_9_installation/red_hat_enterprise_linux-9-performing_an_advanced_rhel_9_installation-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/configuring_basic_system_settings/red_hat_enterprise_linux-9-configuring_basic_system_settings-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/security_hardening/red_hat_enterprise_linux-9-security_hardening-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/composing_a_customized_rhel_system_image/red_hat_enterprise_linux-9-composing_a_customized_rhel_system_image-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/configuring_and_managing_networking/red_hat_enterprise_linux-9-configuring_and_managing_networking-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/pdf/upgrading_from_rhel_8_to_rhel_9/red_hat_enterprise_linux-9-upgrading_from_rhel_8_to_rhel_9-en-us.pdf',
        'https://www.redhat.com/rhdc/managed-files/li-linux-rhel-subscription-guide-detail-639715pr-202312-en_0.pdf'
    ]
    
    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=1275, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

In [ ]:
persist_dir = "db_ansible"

check_file = "False"

path = 'db_ansible/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/containerized_ansible_automation_platform_installation_guide/red_hat_ansible_automation_platform-2.4-containerized_ansible_automation_platform_installation_guide-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/red_hat_ansible_automation_platform_installation_guide/red_hat_ansible_automation_platform-2.4-red_hat_ansible_automation_platform_installation_guide-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/red_hat_ansible_automation_platform_operations_guide/red_hat_ansible_automation_platform-2.4-red_hat_ansible_automation_platform_operations_guide-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/automation_controller_user_guide/red_hat_ansible_automation_platform-2.4-automation_controller_user_guide-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/getting_started_with_ansible_playbooks/red_hat_ansible_automation_platform-2.4-getting_started_with_ansible_playbooks-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/red_hat_ansible_automation_platform/2.4/pdf/deploying_the_red_hat_ansible_automation_platform_operator_on_openshift_container_platform/red_hat_ansible_automation_platform-2.4-deploying_the_red_hat_ansible_automation_platform_operator_on_openshift_container_platform-en-us.pdf'
    ]

    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=1700, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

In [ ]:
persist_dir = "db_openshift"

check_file = "False"

path = 'db_openshift/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.15/pdf/building_applications/openshift_container_platform-4.15-building_applications-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/assisted_installer_for_openshift_container_platform/2024/pdf/installing_openshift_container_platform_with_the_assisted_installer/assisted_installer_for_openshift_container_platform-2024-installing_openshift_container_platform_with_the_assisted_installer-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.15/pdf/storage/openshift_container_platform-4.15-storage-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.15/pdf/cli_tools/openshift_container_platform-4.15-cli_tools-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.15/pdf/virtualization/openshift_container_platform-4.15-virtualization-en-us.pdf',
        'https://access.redhat.com/documentation/en-us/openshift_container_platform/4.15/pdf/windows_container_support_for_openshift/openshift_container_platform-4.15-windows_container_support_for_openshift-en-us.pdf'
    ]

    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=675, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

# Set up the prompts

In [ ]:
# Set up the prompts

rhel_template = """You are an expert in Red Hat Enterprise Linux (RHEL) who retrieves information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

ansible_template = """You are an expert in Ansible Automation Platform (AAP) who retieves information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

openshift_template = """You are an expert in OpenShift Container Platform (OCP), and it's CLI tool oc.\n
You retrieve information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

other_template = """You are an expert in general knowledge.\n
Do not answer the question.\n
Ask how the questions is related a Red Hat product.\n
"""

prompt_templates = [rhel_template, ansible_template, openshift_template, other_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# these lists are used to 
rhel_topics = """Red Hat Enterprise Linux, RHEL"""
ansible_topics = """Ansible Automation Platform, AAP"""
openshift_topics = """OpenShift Container Platform, OCP, oc, vrtctl"""
other_topics = """everything else, satellites, food, fictional characters"""

topic_templates = [rhel_topics, ansible_topics, openshift_topics, other_topics]
topic_embeddings = embeddings.embed_documents(topic_templates)

# Set up some defauls
rag_prompt = ChatPromptTemplate.from_template(ansible_template)

# Define the RAG Chains as functions.

In [ ]:
# the prompt_router function detemines which product the question is most likely about
# and returns the name of the product

def prompt_router(input):
    query_embedding = embeddings.embed_query(input["question"])
    
    similarity = cosine_similarity([query_embedding], topic_embeddings)[0]
    # enable this line if you want to observe similarity scoring
    # print("===== Similarity:   (RHEL       Ansible    OpenShift   Other)\n          Scores: ",similarity,"\n")
    most_similar = prompt_templates[similarity.argmax()]
    
    if most_similar == rhel_template:
        return "rhel"
    elif most_similar == ansible_template:
        return  "ansible"
    elif most_similar == openshift_template:
        return "openshift"
    elif most_similar == other_template:
        return "other"

# Format the source docs forthe LLM
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs)

# Chains

# Process the source_docs to generate the answer
def rag_chain_fn(question):
    rag_from_docs_chain = (
        RunnablePassthrough.assign(
            source_docs=(lambda x: format_docs(x["source_docs"]))
        )
        | rag_prompt
        | model
        | StrOutputParser()
    )
    
    # Retrieve source docs and invoke the last chain.
    rag_chain = (RunnableParallel(
        {
            "source_docs": lambda x: retriever,
            "question": RunnablePassthrough()
        }
    ).assign(answer=rag_from_docs_chain))
    
    # Stream the output
    curr_key = None
    for chunk in rag_chain.stream(question):
        for key in chunk:
            if key == "answer":
                print(chunk[key], end="", flush=True)
    
    # Uncomment this line (and remove the streaming section) to return to normal output.
    #return (rag_chain.invoke(question))


# The router_chain calls the prompt-router function to determine the topic of the question.
def router_chain_fn(question):
    
    router_chain = (
        {"question": RunnablePassthrough()}
        | RunnableLambda(prompt_router)
    )
    
    return (router_chain.invoke(question))

# Main Loop
Here we get the question, set up some variables and route to the correct function to obtain results.

In [ ]:
# Main loop - get the question, figures out the topic, routes to the right db and 
# then works to generate the answer.

question = "What command creates a new user in OpenShift?"

product = router_chain_fn(question)

db_name = "db_" + product

template = eval(product + "_template")

rag_prompt = ChatPromptTemplate.from_template(template)
vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

results = rag_chain_fn(question)

# Uncomment the below lines to revert to normal output.
# answer = results["answer"]
    
# display(Markdown(answer))